In [11]:
from transformers import pipeline
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_openai import OpenAIEmbeddings #for converting chunks into embeddings
from langchain_chroma import Chroma #database for stroring the embeddings
from langchain_huggingface import HuggingFacePipeline

from langchain_community.document_loaders import ImageCaptionLoader

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.retrievers import EnsembleRetriever
from langchain_community.document_loaders.csv_loader import CSVLoader

import pandas as pd
import tempfile
import shutil
import os
# Dictionary to track temporary directories
temp_dirs = []

In [12]:
def img2doc(img_path):
    
    # Create an ImageCaptionLoader instance
    loader = ImageCaptionLoader(img_path)
    # Load the caption as a document
    documents = loader.load()
    
    # Split the document into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    # Create a temporary directory
    temp_dir = tempfile.mkdtemp()
    temp_dirs.append(temp_dir)  # Track temp directory for later cleanup
    
    # Create a new Chroma vectorstore
    vectorstore = Chroma.from_documents(
        documents=splits, 
        embedding=OpenAIEmbeddings(), 
        persist_directory=temp_dir
    )
    retriever = vectorstore.as_retriever(k=1)
    
    return retriever, temp_dir

In [13]:
def fashiondb(dir):
    """
    dir is the directory of the vector DB
    """
    embeddings_used = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorDB = Chroma(persist_directory=dir,embedding_function=embeddings_used)
    retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    return retriever

In [14]:
def cleanup_temp_dir(temp_dir):
    try:
        shutil.rmtree(temp_dir)
        print(f"Temporary directory {temp_dir} cleaned up successfully.")
    except Exception as e:
        print(f"Failed to clean up temporary directory {temp_dir}: {e}")

In [17]:
import os
dir = os.getcwd()
db_dir = os.path.join(dir,"chroma_db")
print(db_dir)

/Users/gabrielaburaglia/Documents/GitHub/closet_care_buddy/chroma_db


In [18]:

def textGeneration_langChain_RAG(img_path):
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    system_prompt = (
        "You are an experienced clothing sylist. "
        "Use the following pieces of retrieved context to answer. "
        "Use one sentence maximum and be as detailed as possible yet concise. "
        "Include the clothing syle (i.e. bohemian, casual, classic, sporty, preppy). "
        "Include how new or worn the item looks to be. "
        "Be confident avoid using the word 'likely'. "
        "\n\n"
        "{context}"
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    caption_retriever,temp_dir = img2doc(img_path)
    reference_retriever = fashiondb(db_dir)

    # initialize the ensemble retriever
    ensemble_retriever = EnsembleRetriever(
    retrievers=[caption_retriever, reference_retriever], weights=[0.5, 0.5]
    )
    

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(ensemble_retriever, question_answer_chain)

    response = rag_chain.invoke({"input": "Describe the piece of clothing in this picture", "context": ensemble_retriever})

    #rag_chain = prompt | llm | StrOutputParser()
    cleanup_temp_dir(temp_dir)

    return response["answer"]

In [19]:
# Path to your image directory or image file
res = textGeneration_langChain_RAG("static/imgs/hippie.png")
print(res)


/Users/gabrielaburaglia/Documents/GitHub/closet_care_buddy/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Temporary directory /var/folders/d2/gls23ldn7jdgvvggh6p22l8c0000gn/T/tmpka_hulq1 cleaned up successfully.
The image likely features a pair of cowboy boots, which are typically associated with a western style and appear to be new.
